In [43]:
from __future__ import print_function
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import random
import torch
import math
from torch.autograd import Variable

In [44]:
class ResNet(nn.Module):
    def __init__(self, game, num_resBlocks, num_hidden, device):
        super().__init__()
        
        self.device = device
        self.startBlock = nn.Sequential(
            nn.Conv2d(3, num_hidden, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_hidden),
            nn.ReLU()
        )
        
        self.backBone = nn.ModuleList(
            [ResBlock(num_hidden) for i in range(num_resBlocks)]
        )
        
        self.policyHead = nn.Sequential(
            nn.Conv2d(num_hidden, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * game.row_count * game.column_count, game.action_size)
        )
        
        self.valueHead = nn.Sequential(
            nn.Conv2d(num_hidden, 3, kernel_size=3, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3 * game.row_count * game.column_count, 1),
            nn.Tanh()
        )
        
        self.to(device)
        
    def forward(self, x):
        x = self.startBlock(x)
        for resBlock in self.backBone:
            x = resBlock(x)
        policy = self.policyHead(x)
        value = self.valueHead(x)
        return policy, value
        
        
class ResBlock(nn.Module):
    def __init__(self, num_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_hidden)
        self.conv2 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_hidden)
        
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x
        

In [45]:
class Node:
    def __init__(self, game, args, state,parent=None, action_taken=None, prior=0, visit_count=0):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        self.prior = prior
        
        self.children = []
        
        self.visit_count = visit_count
        self.value_sum = 0
        
    def is_fully_expanded(self):
        return len(self.children) > 0
    
    def select(self):
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
                
        return best_child
    
    def get_ucb(self, child):
        if child.visit_count == 0:
            q_value = 0
        else:
            q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * (math.sqrt(self.visit_count) / (child.visit_count + 1)) * child.prior
    
    def expand(self, policy):
        child=None
        for action, prob in enumerate(policy):

            if prob > 0:
                child_state = self.state.copy()
                child_state = self.game.get_next_state(child_state, action, 1)
                child_state = self.game.change_perspective(child_state, player=-1)

                child = Node(self.game, self.args, child_state, self, action, prob)
                self.children.append(child)
        if(child is None):
            child_state = self.state.copy()
            child_state = self.game.change_perspective(child_state, player=-1)

            child = Node(self.game, self.args, child_state, self, action, prob)
            self.children.append(child)   
        return child
            
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTSParallel:
    def __init__(self, game, args, model):
        self.game = game
        self.args = args
        self.model = model
        
    @torch.no_grad()
    def search(self, state,spGames):
        policy, _ = self.model(
            torch.tensor(self.game.get_encoded_state(state), device=self.model.device)
        )
        policy = torch.softmax(policy, axis=1).cpu().numpy()
        policy = (1 - self.args['dirichlet_epsilon']) * policy + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size, size=policy.shape[0])
        
        
        for i, spg in enumerate(spGames):
            spg_policy = policy[i]
            valid_moves = self.game.get_valid_moves(state[i],1)
            spg_policy *= valid_moves
            if( np.sum(spg_policy)!=0):
                spg_policy /= np.sum(spg_policy)
            spg.root = Node(self.game, self.args, state[i], visit_count=1)
            spg.root.expand(spg_policy)
        
        for search in range(self.args['num_searches']):
            for spg in spGames:
                spg.node = None
                node = spg.root
                while node.is_fully_expanded():
                    node = node.select()

                value, is_terminal = self.game.get_value_and_terminated(node.state)
                value = self.game.get_opponent_value(value)

                if is_terminal:
                    node.backpropagate(value)
                    
                else:
                    spg.node = node
                    
            expandable_spGames = [mappingIdx for mappingIdx in range(len(spGames)) if spGames[mappingIdx].node is not None]
            
            if len(expandable_spGames) > 0:
                state = np.stack([spGames[mappingIdx].node.state for mappingIdx in expandable_spGames])
                
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(state), device=self.model.device)
                )
                policy = torch.softmax(policy, axis=1).cpu().numpy()
                value = value.cpu().numpy()
            
            for i, mappingIdx in enumerate(expandable_spGames):
                node = spGames[mappingIdx].node
                spg_policy, spg_value = policy[i], value[i]
                
                valid_moves = self.game.get_valid_moves(node.state,1)
                spg_policy *= valid_moves
                if np.sum(spg_policy)!=0:
                    spg_policy /= np.sum(spg_policy)

                node.expand(spg_policy)
                node.backpropagate(spg_value)  
            
            

In [46]:
class MCTS:
    def __init__(self, game, args, model):
        self.game = game
        self.args = args
        self.model = model
        
    @torch.no_grad()
    def search(self, state):
        #aqui buscar tabuleiro antigo
        root = Node(self.game, self.args, state, visit_count=1)
        
        policy, _ = self.model(
            torch.tensor(self.game.get_encoded_state(state), device=self.model.device).unsqueeze(0)
        )
        policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy()
        policy = (1 - self.args['dirichlet_epsilon']) * policy + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size)
        #por tabuleiro,dentro tornar em gamestate
        valid_moves = self.game.get_valid_moves(root.state,1)
        policy *= valid_moves
        policy /= np.sum(policy)
        root.expand(policy)

        
        for search in range(self.args['num_searches']):
            node = root
            while node.is_fully_expanded():
                node = node.select()
            value, is_terminal = self.game.get_value_and_terminated(node.state)
            value = self.game.get_opponent_value(value)

            if not is_terminal:
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(node.state), device=self.model.device).unsqueeze(0)
                )
                policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy()
                
                valid_moves = self.game.get_valid_moves(node.state,1)
                policy *= valid_moves
                policy /= np.sum(policy)
                
                pre=None
                value = value.item()
                
                node.expand(policy)
                
            node.backpropagate(value)    
            
            
        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs

In [47]:
from tqdm.notebook import trange
class AlphaZeroParallel:
    def __init__(self, model, optimizer, game, args):
        self.model = model
        self.optimizer = optimizer
        self.game = game
        self.args = args
        self.mcts = MCTSParallel(game, args, model)
        
    def selfPlay(self):
        return_memory = []
        player = 1
        spGames = [SPG(self.game) for spg in range(self.args['num_parallel_games'])]
        
        while len(spGames) > 0:
            print(len(spGames))
            states = np.stack([spg.state for spg in spGames])
            neutral_states = self.game.change_perspective(states, player)
            #passar variavel de passar e tabuleiro antigo##########################################
            action_probs = self.mcts.search(neutral_states,spGames)
            for i in range(len(spGames))[::-1]:
                spg = spGames[i]
                
                action_probs = np.zeros(self.game.action_size)
                for child in spg.root.children:
                    action_probs[child.action_taken] = child.visit_count
                
                
                if np.sum(action_probs)!=0:
                    action_probs /= np.sum(action_probs)

                spg.memory.append((spg.root.state, action_probs, player))

                temperature_action_probs = action_probs ** (1 / self.args['temperature'])
                if np.sum(temperature_action_probs)!=0:
                    temperature_action_probs /= np.sum(temperature_action_probs)
                action = np.random.choice(self.game.action_size, p=temperature_action_probs) # Divide temperature_action_probs with its sum in case of an error

                spg.state = self.game.get_next_state(spg.state, action, player)
                value, is_terminal = self.game.get_value_and_terminated(spg.state)
                if is_terminal:
                    for hist_neutral_state, hist_action_probs, hist_player in spg.memory:
                        hist_outcome = value if hist_player == player else self.game.get_opponent_value(value)
                        return_memory.append((
                            self.game.get_encoded_state(hist_neutral_state),
                            hist_action_probs,
                            hist_outcome
                        ))
                    del spGames[i]
            
            player = self.game.get_opponent(player)
        
        return return_memory
                
    def train(self, memory):
        random.shuffle(memory)
        for batchIdx in range(0, len(memory), self.args['batch_size']):
            sample = memory[batchIdx:min(len(memory) - 1, batchIdx + self.args['batch_size'])] # Change to memory[batchIdx:batchIdx+self.args['batch_size']] in case of an error
            state, policy_targets, value_targets = zip(*sample)
            
            state, policy_targets, value_targets = np.array(state), np.array(policy_targets), np.array(value_targets).reshape(-1, 1)
            
            state = torch.tensor(state, dtype=torch.float32, device=self.model.device)
            policy_targets = torch.tensor(policy_targets, dtype=torch.float32, device=self.model.device)
            value_targets = torch.tensor(value_targets, dtype=torch.float32, device=self.model.device)
            
            out_policy, out_value = self.model(state)
            
            policy_loss = F.cross_entropy(out_policy, policy_targets)
            value_loss = F.mse_loss(out_value, value_targets)
            loss = policy_loss + value_loss
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
    
    def learn(self):
        for iteration in range(self.args['num_iterations']):
            memory = []
            
            self.model.eval()
            for selfPlay_iteration in trange(self.args['num_selfPlay_iterations'] // self.args['num_parallel_games']):
                memory += self.selfPlay()
                
            self.model.train()
            for epoch in trange(self.args['num_epochs']):
                self.train(memory)
            
            torch.save(self.model.state_dict(), f"model_{iteration}_{self.game}.pt")
            torch.save(self.optimizer.state_dict(), f"optimizer_{iteration}_{self.game}.pt")
class SPG:
    def __init__(self, game):
        self.state = game.get_initial_state()
        self.memory = []
        self.root = None
        self.node = None

In [48]:
import atax
class Atax():
    def __init__(self,n):
        self.row_count = n
        self.column_count = n
        self.action_size = n*n*24
        self.num_to_pos= { 0 :(-2,-2),1 :(-2,-1),2 :(-2,0),3:(-2,1),4:(-2,2),
                            5:(-1,-2),6:(-1,-1),7:(-1,0),8:(-1,1),9:(-1,2),
                            10:(0,-2),11:(0,-1),12:(0,1),13:(0,2),
                            14:(1,-2),15:(1,-1),16:(1,0),17:(1,1),18:(1,2),
                            19:(2,-2),20:(2,-1),21:(2,0),22:(2,1),23:(2,2)}
        self.pos_to_num= {v: k for k, v in self.num_to_pos.items()}
    def __repr__(self):
        return "Atax"
        
    def get_initial_state(self):
        b=np.zeros((self.row_count, self.column_count))
        b[0][0]=1
        b[0][self.column_count-1]=-1
        b[self.row_count-1][self.column_count-1]=1
        b[self.row_count-1][0]=-1
        return b
    
    def get_next_state(self, state, action, player):
        #para o caso de passar
        if action ==-1:
            return state
        b = atax.State(state,player)
        val = action//24
        xi= val//self.column_count
        yi=val%self.column_count
        pos= self.num_to_pos[action%24]
        #por a ir ao dicionario
        xf = xi+pos[0]
        yf = yi+pos[1]
        ty=1
        if abs(pos[0])==2 or abs(pos[1])==2:
            ty=2
        move= atax.Move(xi,yi,xf,yf,player,ty)
        boa=b.execute_move(move)
        return boa
    
    def get_valid_moves(self, state,player):
        valid_moves = [0] * self.action_size
        b = atax.State(state,player)
        possi=b.available_moves(1)
        for i in possi:
            dicti=(i.xf - i.xi,i.yf-i.yi)
            action = self.pos_to_num[dicti] + (i.xi*self.column_count + i.yi)*24
            #por a ir ao dicionario
            valid_moves[action]=1
        return valid_moves
    
    def get_value_and_terminated(self, state):
        value,terminated = self.winner(state)
        return value, terminated
    
    def winner(self, state):
        b=atax.State(state,1)
        pecas= self.count(state)
        b1=b.available_moves(1).size
        b_1=b.available_moves(-1).size
        if pecas[0] == 0:
            if pecas[1] == pecas[2]:
                return 0,True
            elif pecas[1] < pecas[-1]:
                return -1,True
            else: return 1,True
        elif pecas[1] == 0:
            return -1,True
        elif pecas[-1] == 0:
            return 1,True
        
        b1=b.available_moves(1).size
        b_1=b.available_moves(-1).size
            
        if b1==0:
            if pecas[1] < pecas[-1] + pecas[0]:
                return -1,True
            else: return 1,True
        if b_1==0:
            if pecas[-1] < pecas[1] + pecas[0]:
                return 1,True
            else: return -1,True
        return 0,False
    
    def count(self,state):
        """counts pieces"""
        pecas=[0,0,0]
        for i in range(0, self.row_count):
            for j in range(0, self.column_count):
                if state[i][j] == 0:
                    pecas[0]+=1
                elif state[i][j] == 1:
                    pecas[1] +=1
                else:
                    pecas[-1] +=1
        return pecas
    
    def get_opponent(self, player):
        return -player
    
    def get_opponent_value(self, value):
        return -value
    
    def change_perspective(self, state, player):
        return state * player
    
    def get_encoded_state(self, state):
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1)
        ).astype(np.float32)
        
        if len(state.shape) == 3:
            encoded_state = np.swapaxes(encoded_state, 0, 1)
        
        return encoded_state

In [49]:
game = Atax(6)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNet(game, 9, 128, device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

args = {
    'C': 2,
    'num_searches': 200,
    'num_iterations': 1,
    'num_selfPlay_iterations': 40,
    'num_parallel_games': 20,
    'num_epochs': 4,
    'batch_size': 128,
    'temperature': 1.25,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3
}

alphaZero = AlphaZeroParallel(model, optimizer, game, args)
alphaZero.learn()

  0%|          | 0/2 [00:00<?, ?it/s]

20
20
20
20
20
20
20
18
16
15
15
15
15
15
15
14
14
13
13
13
12
11
11
11
10
9
8
8
7
7
7
7
7
7
5
4
2
2
20
20
20
20
20
